In [7]:
import math

def calculate_mde_from_sample_size(sample_size, baseline_rate):
    """
    Calculate the Minimum Detectable Effect (MDE) using the simplified formula with a factor of 16.
    
    :param sample_size: The number of users per group (control or variant).
    :param baseline_rate: The baseline conversion rate (proportion, e.g., 0.10 for 10%).
    :return: The Minimum Detectable Effect (MDE) as a percentage.
    """
    
    # Calculate the standard deviation (σ) based on the baseline rate
    sigma = math.sqrt(baseline_rate * (1 - baseline_rate))

    # Simplified formula to calculate the MDE
    mde = math.sqrt((16 * sigma ** 2) / sample_size)

    # Convert the MDE to a percentage and return
    return mde * 100

# Example usage
sample_size = 5000
baseline_rate = 0.10  # 10% baseline conversion rate

mde = calculate_mde_from_sample_size(sample_size, baseline_rate)
print(f"The Minimum Detectable Effect (MDE) is: {mde:.2f}%")

The Minimum Detectable Effect (MDE) is: 1.70%


In [9]:
import math
from scipy.stats import norm

# Function to calculate sample size based on variance, MDE, alpha, and beta (power)
def calculate_sample_size(variance, mde, alpha=0.05, power=0.8):
    # Calculate Z-scores based on alpha and beta (for power)
    z_alpha = norm.ppf(1 - alpha / 2)  # Two-tailed test (1 - alpha/2)
    z_beta = norm.ppf(power)  # Power (1 - beta)

    # Calculate sample size using the formula: n = (Zα/2 + Zβ)^2 * σ^2 / δ^2
    sample_size = ((z_alpha + z_beta) ** 2) * (variance / (mde ** 2))

    return math.ceil(sample_size)  # Round up to the nearest integer

# Function to calculate Dunnett's critical value for multiple comparisons
def calculate_dunnett_critical_value(confidence_level, num_comparisons):
    # For Dunnett's test with one-sided adjustment, use the student t-distribution
    critical_value = norm.ppf(confidence_level)  # One-sided normal distribution

    return critical_value

# Main function to calculate MDE for one-sided tests
def calculate_mde_one_sided(weekly_traffic, weekly_conversions, confidence_level, power, num_variants, weeks=[1, 2, 3, 4, 5, 6], use_dunnetts_adjustment=False):
    # Total number of groups including control
    total_groups = num_variants  # Includes control

    # Number of test variants (exclude control)
    test_variants = num_variants - 1  # Exclude control group

    # Adjust confidence level using Dunnett's adjustment if needed
    adjusted_confidence_level = confidence_level
    if use_dunnetts_adjustment and test_variants > 1:
        adjusted_confidence_level = calculate_dunnett_critical_value(confidence_level, test_variants)

    # Z-scores for one-sided test
    z_alpha = norm.ppf(1 - (1 - adjusted_confidence_level))  # One-sided test
    z_beta = norm.ppf(power)

    baseline_cr = weekly_conversions / weekly_traffic  # Baseline conversion rate
    variance_oec = baseline_cr * (1 - baseline_cr)  # Variance of the outcome metric (OEC)

    mde_list = []

    for w in weeks:
        # Step 1: Calculate the number of visitors per variant for each week
        visitors_per_variant = (weekly_traffic * w) / num_variants  # Divide by total groups (including control)

        # Step 2: Calculate the absolute MDE based on variance and Z-scores using visitors per variant
        mde_absolute = (z_alpha + z_beta) * math.sqrt(variance_oec / visitors_per_variant)

        # Step 3: Calculate the relative MDE as a percentage
        relative_mde = (mde_absolute / baseline_cr) * 100

        relative_mde = calculate_mde_from_sample_size(visitors_per_variant, baseline_cr)

        # Step 4: Recalculate the sample size based on the calculated MDE
        recalculated_sample_size = calculate_sample_size(variance_oec, mde_absolute, confidence_level, power)

        # Print the recalculated sample size and the original sample size for comparison
        print(f"Week {w} -> Original Sample Size: {visitors_per_variant}, Recalculated Sample Size: {recalculated_sample_size}")

        # Append week, absolute MDE, relative MDE, visitors per test variant, and recalculated sample size
        mde_list.append({
            'week': w,
            'mde_absolute': mde_absolute,
            'relative_mde': relative_mde,
            'total_users_per_variant': visitors_per_variant,
            'recalculated_sample_size': recalculated_sample_size,
        })

    return mde_list


# Example usage
weekly_traffic = 10000
weekly_conversions = 200
confidence_level = 0.95
power = 0.8
num_variants = 2  # Includes control
weeks = [1, 2, 3, 4, 5, 6]

results = calculate_mde_one_sided(weekly_traffic, weekly_conversions, confidence_level, power, num_variants, weeks)

for result in results:
    print(f"Week: {result['week']}, Relative MDE: {result['relative_mde']:.2f}%, Visitors per Variant: {result['total_users_per_variant']}, Recalculated Sample Size: {result['recalculated_sample_size']}")

Week 1 -> Original Sample Size: 5000.0, Recalculated Sample Size: 662
Week 2 -> Original Sample Size: 10000.0, Recalculated Sample Size: 1323
Week 3 -> Original Sample Size: 15000.0, Recalculated Sample Size: 1985
Week 4 -> Original Sample Size: 20000.0, Recalculated Sample Size: 2646
Week 5 -> Original Sample Size: 25000.0, Recalculated Sample Size: 3307
Week 6 -> Original Sample Size: 30000.0, Recalculated Sample Size: 3969
Week: 1, Relative MDE: 0.79%, Visitors per Variant: 5000.0, Recalculated Sample Size: 662
Week: 2, Relative MDE: 0.56%, Visitors per Variant: 10000.0, Recalculated Sample Size: 1323
Week: 3, Relative MDE: 0.46%, Visitors per Variant: 15000.0, Recalculated Sample Size: 1985
Week: 4, Relative MDE: 0.40%, Visitors per Variant: 20000.0, Recalculated Sample Size: 2646
Week: 5, Relative MDE: 0.35%, Visitors per Variant: 25000.0, Recalculated Sample Size: 3307
Week: 6, Relative MDE: 0.32%, Visitors per Variant: 30000.0, Recalculated Sample Size: 3969


In [11]:
import math
import math

def calculate_sigma(baseline_rate):
    """
    Calculate the standard deviation (σ) based on the baseline conversion rate.
    
    :param baseline_rate: The baseline conversion rate (proportion, e.g., 0.10 for 10%).
    :return: The standard deviation (σ).
    """
    # Calculate the variance σ²
    variance = baseline_rate * (1 - baseline_rate)
    
    # Calculate the standard deviation (σ)
    sigma = math.sqrt(variance)
    
    return sigma

# Example usage
baseline_rate = 0.10  # 10% baseline conversion rate

sigma = calculate_sigma(baseline_rate)
print(f"Standard deviation (σ): {sigma:.4f}")

def calculate_mde(sample_size, sigma):
    """
    Calculate the Minimum Detectable Effect (MDE) using the correct formula.
    
    :param sample_size: The number of users per group (control or variant).
    :param sigma: The standard deviation (σ) for the baseline conversion rate.
    :return: The Minimum Detectable Effect (MDE) as a percentage.
    """
    
    # Use the correct formula for MDE without an extra square root
    mde = math.sqrt((16 * sigma ** 2) / sample_size)

    # Convert the MDE to a percentage and return
    return mde * 100

# Example usage
sample_size = 5000
baseline_rate = 0.10  # 10% baseline conversion rate

# Calculate the standard deviation σ based on the baseline conversion rate
sigma = calculate_sigma(baseline_rate)

# Calculate the MDE
mde = calculate_mde(sample_size, sigma)
print(f"The Minimum Detectable Effect (MDE) is: {mde:.2f}%")

Standard deviation (σ): 0.3000
The Minimum Detectable Effect (MDE) is: 1.70%


## Evans calculator

In [44]:
import math
from scipy.stats import norm

def ppnd(p):
    """
    Compute the inverse of the cumulative distribution function (CDF) for the normal distribution.
    This corresponds to the ppnd function in the JavaScript code.
    """
    return norm.ppf(p)

def calculate_sample_size(alpha, power_level, conversion_rate, min_effect, is_relative=True, one_sided=True):
    """
    Calculate the required sample size based on the input parameters.
    
    Parameters:
    - alpha: Significance level (Type I error rate)
    - power_level: Power level (1 - Type II error rate)
    - conversion_rate: The base conversion rate (p)
    - min_effect: The minimum detectable effect (MDE)
    - is_relative: Boolean indicating if the MDE is relative or absolute
    - one_sided: Boolean indicating if the test is one-sided or two-sided
    
    Returns:
    - The required sample size for each group.
    """
    
    # Adjust for relative or absolute difference
    delta = min_effect * conversion_rate if is_relative else min_effect
    
    # Ensure the conversion rate is within the correct range
    if conversion_rate > 0.5:
        conversion_rate = 1.0 - conversion_rate
    
    # Critical values for alpha and power level (one-sided or two-sided)
    t_alpha = ppnd(1.0 - alpha) if one_sided else ppnd(1.0 - alpha / 2)  # Z value for significance level
    t_beta = ppnd(power_level)  # Z value for power level
    
    # Standard deviations for control and test groups
    sd1 = math.sqrt(2 * conversion_rate * (1.0 - conversion_rate))  # control group standard deviation
    sd2 = math.sqrt(conversion_rate * (1.0 - conversion_rate) + (conversion_rate + delta) * (1.0 - (conversion_rate + delta)))  # test group standard deviation
    
    # Sample size calculation
    sample_size = (t_alpha * sd1 + t_beta * sd2) ** 2 / (delta ** 2)
    
    return math.ceil(sample_size)  # Return the ceiling of the sample size to ensure full participants

# Example usage:
alpha_level = 0.05  # 5% significance level
power_level = 0.80  # 80% power level
conversion_rate = 0.30  # Base conversion rate 30%
min_effect = 0.4  # Minimum detectable effect (5%)
is_relative = True  # Use relative difference
one_sided = True  # Dont use one-sided test by default

sample_size = calculate_sample_size(alpha_level, power_level, conversion_rate, min_effect, is_relative, one_sided)
print(f"Required sample size per group for one-sided: {sample_size}")

one_sided = False  # Use two-sided test
sample_size_twosided = calculate_sample_size(alpha_level, power_level, conversion_rate, min_effect, is_relative, one_sided)
print(f"Required sample size per group for two-sided: {sample_size_twosided}")

Required sample size per group for one-sided: 186
Required sample size per group for two-sided: 235


In [41]:
import math
from scipy.stats import norm

def ppnd(p):
    """
    Compute the inverse of the cumulative distribution function (CDF) for the normal distribution.
    This corresponds to the ppnd function in the JavaScript code.
    """
    return norm.ppf(p)

def calculate_mde(alpha, power_level, conversion_rate, num_subjects, is_relative=True, one_sided=True):
    """
    Calculate the minimum detectable effect (MDE) based on the input parameters.
    
    Parameters:
    - alpha: Significance level (Type I error rate)
    - power_level: Power level (1 - Type II error rate)
    - conversion_rate: The base conversion rate (p)
    - num_subjects: The number of subjects for each group.
    - is_relative: Boolean indicating if the MDE is relative or absolute
    - one_sided: Boolean indicating if the test is one-sided or two-sided
    
    Returns:
    - The minimum detectable effect (MDE) for the given sample size.
    """
    
    # Ensure the conversion rate is within the correct range
    if conversion_rate > 0.5:
        conversion_rate = 1.0 - conversion_rate
    
    # Critical values for alpha and power level (one-sided or two-sided)
    t_alpha = ppnd(1.0 - alpha) if one_sided else ppnd(1.0 - alpha / 2)  # Z value for significance level
    t_beta = ppnd(power_level)  # Z value for power level
    
    # Standard deviations for control and test groups
    sd1 = math.sqrt(2 * conversion_rate * (1.0 - conversion_rate))  # control group standard deviation
    # The standard deviation for the test group will use delta, which we want to solve for

    # Rearranging the sample size formula to solve for delta
    # num_subjects = (t_alpha * sd1 + t_beta * sd2) ** 2 / delta ** 2
    # delta = sqrt((t_alpha * sd1 + t_beta * sd2) ** 2 / num_subjects)

    # Calculate the minimum detectable effect (MDE)
    # We assume that initially delta is small so the variance change is negligible for sd2,
    # so sd2 ≈ sd1. If sd1 and sd2 are very different, further refinement might be needed.
    delta = math.sqrt((t_alpha * sd1 + t_beta * sd1) ** 2 / num_subjects)

    # If the MDE is relative, we return it relative to the conversion rate
    return delta / conversion_rate if is_relative else delta

# Example to calculate the MDE with given parameters
alpha = 0.05
power_level = 0.8
conversion_rate = 0.3
num_subjects = 235
is_relative = True
one_sided = False

mde = calculate_mde(alpha, power_level, conversion_rate, num_subjects, is_relative, one_sided)
print(f"The minimum detectable effect (MDE) is: {mde}")

The minimum detectable effect (MDE) is: 0.3947965025334013


In [48]:
import numpy as np

def test_functions():
    """
    Test that num_subjects and calculate_mde produce coherent output for a range of conversion rates.
    """
    alpha = 0.05
    power_level = 0.8
    min_effect = 0.02
    is_relative = True
    one_sided = True
    tolerance = 1e-3  # Tolerance for floating-point precision

    # Test a range of conversion rates from 0.01 to 0.5
    conversion_rates = np.linspace(0.01, 0.5, 10)

    for conversion_rate in conversion_rates:
        # Step 1: Calculate sample size based on the MDE
        calculated_sample_size = calculate_sample_size(alpha, power_level, conversion_rate, min_effect, is_relative, one_sided)

        # Step 2: Calculate MDE from the sample size calculated
        calculated_mde = calculate_mde(alpha, power_level, conversion_rate, calculated_sample_size, is_relative, one_sided)

        # Check if the MDE obtained from the sample size calculation matches the original input MDE
        if np.abs(min_effect - calculated_mde) <= tolerance:
            print(f"Test passed for conversion rate {conversion_rate:.2f}! The MDE calculated from the sample size matches the original MDE.")
        else:
            print(f"Test failed for conversion rate {conversion_rate:.2f}! Expected MDE: {min_effect}, but got {calculated_mde}.")

# Run the test
test_functions()

Test passed for conversion rate 0.01! The MDE calculated from the sample size matches the original MDE.
Test passed for conversion rate 0.06! The MDE calculated from the sample size matches the original MDE.
Test passed for conversion rate 0.12! The MDE calculated from the sample size matches the original MDE.
Test passed for conversion rate 0.17! The MDE calculated from the sample size matches the original MDE.
Test passed for conversion rate 0.23! The MDE calculated from the sample size matches the original MDE.
Test passed for conversion rate 0.28! The MDE calculated from the sample size matches the original MDE.
Test passed for conversion rate 0.34! The MDE calculated from the sample size matches the original MDE.
Test passed for conversion rate 0.39! The MDE calculated from the sample size matches the original MDE.
Test passed for conversion rate 0.45! The MDE calculated from the sample size matches the original MDE.
Test passed for conversion rate 0.50! The MDE calculated from th